In [11]:
from mesa import Agent
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
from mesa.batchrunner import BatchRunner
from collections import defaultdict
import itertools
import numpy as np
import pandas as pd

In [2]:
class RandomActivationByBreed(RandomActivation):
    def __init__(self,model):
        super().__init__(model)
        self.agents_by_breed=defaultdict(dict)
    def add(self,agent):
        self._agents[agent.unique_id]=agent
        agent_class=type(agent)
        self.agents_by_breed[agent_class][agent.unique_id]=agent
    def remove(self,agent):
        self._agents[agent.unique_id]
        agent_class=type(agent)
        del self.agents_by_breed[agent_class][agent.unique_id]
    def step(self,by_breed=True):
        if by_breed:
            for agent_class in self.agents_by_breed:
                self.step_breed(agent_class)
            self.steps+=1
            self.time+=1
        else:
            super().step()
    def step_breed(self,breed):
        agent_keys=list(self.agents_by_breed[breed].keys())
        self.model.random.shuffle(agent_keys)
        for agent_key in agent_keys:
            self.agents_by_breed[breed][agent_key].step()
    def get_breed_count(self,breed_class):
        return len(self.agents_by_breed[breed_class].values())

In [3]:
class RandomWalker(Agent):
    grid=None
    x=None
    y=None
    moore=True
    def __init__(self,unique_id,pos,model,moore=True):
        super().__init__(unique_id,model)
        self.pos=pos
        self.moore=moore
    def random_move(self):
        next_moves=self.model.grid.get_neighborhood(self.pos,self.moore,True)
        next_move=self.random.choice(next_moves)
        self.model.grid.move_agent(self,next_move)

In [4]:
class Sheep(RandomWalker):
    energy=None
    def __init__(self,unique_id,pos,model,moore,energy=None):
        super().__init__(unique_id,pos,model,moore=moore)
        self.energy=energy
    def step(self):
        self.random_move()
        self.energy-=1
        this_cell=self.model.grid.get_cell_list_contents([self.pos,])
        grass_patch=[obj for obj in this_cell if isinstance(obj,GrassPatch)][0]
        if grass_patch.fully_grown:
            self.energy+=self.model.sheep_gain_from_food
            grass_patch.fully_grown=False
        if self.energy<0:
            self.model.grid._remove_agent(self.pos,self) #그리드에서 agent를 제거한다.
            self.model.schedule.remove(self) #활동 스케쥴에서 agent를 제거한다.
        else:
            if self.random.random() < self.model.sheep_reproduce:
                self.energy/=2
                lamb=Sheep(
                    self.model.next_id(),self.pos,self.model,self.moore,self.energy
                    )
                self.model.grid.place_agent(lamb,lamb.pos)
                self.model.schedule.add(lamb)

In [5]:
class Wolf(RandomWalker):
    energy=None
    def __init__(self,unique_id,pos,model,moore,energy=None):
        super().__init__(unique_id,pos,model,moore=moore)
        self.energy=energy
    def step(self):
        self.random_move()
        self.energy-=1
        this_cell=self.model.grid.get_cell_list_contents([self.pos,])
        sheep=[obj for obj in this_cell if isinstance(obj,Sheep)]
        if len(sheep)>0:
            sheep_to_eat=self.random.choice(sheep)
            self.energy+=self.model.wolf_gain_from_food
            self.model.grid._remove_agent(self.pos,sheep_to_eat)
            self.model.schedule.remove(sheep_to_eat)
        if self.energy<0:
            self.model.grid._remove_agent(self.pos,self)
            self.model.schedule.remove(self)
        else:
            if self.random.random()<self.model.wolf_reproduce:
                self.energy/=2
                cub=Wolf(
                    self.model.next_id(),self.pos,self.model,self.moore,self.energy
                    )
                self.model.grid.place_agent(cub,cub.pos)
                self.model.schedule.add(cub)

In [6]:
class GrassPatch(Agent):
    def __init__(self,unique_id,pos,model,fully_grown,countdown):
        super().__init__(unique_id,model)
        self.fully_grown=fully_grown
        self.countdown=countdown
        self.pos=pos
    def step(self):
        if not self.fully_grown:
            if self.countdown<=0:
                self.fully_grown=True
                self.coutdown=self.model.grass_regrowth_time
            else:
                self.countdown-=1

In [15]:
class WolfSheep(Model):
    height=20
    width=20
    initial_sheep=100
    initial_wolves=50
    sheep_reproduce=0.04
    wolf_reproduce=0.05
    wolf_gain_from_food=20
    grass_regrowth_time=30
    sheep_gain_from_food=4
    id_gen=itertools.count(1)
    def __init__(self,
                height,
                width,
                initial_sheep,
                initial_wolves,
                sheep_reproduce,
                wolf_reproduce,
                wolf_gain_from_food,
                grass_regrowth_time,
                sheep_gain_from_food):
        super().__init__()
        self.unique_id=next(self.id_gen)
        self.height=height
        self.width=width
        self.initial_sheep=initial_sheep
        self.initial_wolves=initial_wolves
        self.sheep_reproduce=sheep_reproduce
        self.wolf_reproduce=wolf_reproduce
        self.wolf_gain_from_food=wolf_gain_from_food
        self.grass_regrowth_time=grass_regrowth_time
        self.sheep_gain_from_food=sheep_gain_from_food
        self.schedule=RandomActivationByBreed(self)
        self.grid=MultiGrid(self.height,self.width,torus=True)
        self.datacollector=DataCollector(
                model_reporters={
                    "Wolves":lambda m:m.schedule.get_breed_count(Wolf),
                    "Sheep":lambda m:m.schedule.get_breed_count(Sheep),
                    "ModelParams":lambda m:{'sheep_reproduce':m.sheep_reproduce,'wolf_reproduce':m.wolf_reproduce},
                    "Run":lambda m:m.unique_id
                }
            )
        for i in range(self.initial_sheep):
            x=self.random.randrange(self.width)
            y=self.random.randrange(self.height)
            energy=self.random.randrange(2*self.sheep_gain_from_food)
            sheep=Sheep(self.next_id(),(x,y),self,True,energy)
            self.grid.place_agent(sheep,sheep.pos)
            self.schedule.add(sheep)
        for i in range(self.initial_wolves):
            x=self.random.randrange(self.width)
            y=self.random.randrange(self.height)
            energy=self.random.randrange(2*self.wolf_gain_from_food)
            wolf=Wolf(self.next_id(),(x,y),self,True,energy)
            self.grid.place_agent(wolf,wolf.pos)
            self.schedule.add(wolf)
        for agent,x,y in self.grid.coord_iter():
            fully_grown=self.random.choice([True,False])
            if fully_grown:
                countdown=self.grass_regrowth_time
            else:
                countdown=self.random.randrange(self.grass_regrowth_time)
            patch=GrassPatch(self.next_id(),(x,y),self,fully_grown,countdown)
            self.grid.place_agent(patch,(x,y))
            self.schedule.add(patch)
        self.running=True
        self.datacollector.collect(self)
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
    def run_model(self,step_count=200):
        print("Initial number wolves:",self.schedule.get_breed_count(Wolf))
        print("Initial number sheep:",self.schedule.get_breed_count(Sheep))
        for i in range(step_count):
            self.step()
        print("")
        print("Final number wolves:",self.schedule.get_breed_count(Wolf))
        print("Final numbe sheep:",self.schedule.get_breed_count(Sheep))

In [18]:
fixed_params={'width':WolfSheep.width,
             'height':WolfSheep.height,
             'initial_sheep':WolfSheep.initial_sheep,
             'initial_wolves':WolfSheep.initial_wolves,
              'wolf_gain_from_food':WolfSheep.wolf_gain_from_food,
              'grass_regrowth_time':WolfSheep.grass_regrowth_time,
              'sheep_gain_from_food':WolfSheep.sheep_gain_from_food,
             }
variable_params={
    'sheep_reproduce':[WolfSheep.sheep_reproduce,WolfSheep.sheep_reproduce+0.01],
    'wolf_reproduce':[WolfSheep.wolf_reproduce-0.01,WolfSheep.wolf_reproduce],
}
batch_run=BatchRunner(
    WolfSheep,
    variable_params, #세팅의 수는 49개 range(10,500,10), 10에서 490까지
    fixed_params,
    iterations=1, #동일한 세팅을 몇 번 반복할까? 49*5=245회의 시뮬레이션 수행
    max_steps=500,
    model_reporters={'DataCollector':lambda m:m.datacollector}
)
batch_run.run_all()

4it [00:38, 11.67s/it]


In [19]:
br_df=batch_run.get_model_vars_dataframe()
br_step_data=pd.DataFrame()
for i in range(len(br_df['DataCollector'])):
    if isinstance(br_df['DataCollector'][i],DataCollector):
        i_run_data=br_df['DataCollector'][i].get_model_vars_dataframe()
        br_step_data=br_step_data.append(i_run_data,ignore_index=True)
br_step_data.to_csv("WolfSheep_Step_Data.csv")